# Imports

In [ ]:
import os
import tensorflow as tf
import onnx
from onnx_tf.backend import prepare
import torch
import torch.onnx
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from PIL import Image

## Processing Data

### Image Transforms

In [ ]:
resolution = (256, 256) # Resolution of images
# For training (with augmentation)
train_transform = transforms.Compose([
    transforms.Resize(resolution),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# For validation and testing (without augmentation)
transform = transforms.Compose([
    transforms.Resize(resolution),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### Load Datasets

In [ ]:
# Load Dataset with validation/test transform
dataset = ImageFolder(root='../data/filtered', transform=transform)

# Determine the number of classes
num_classes = len(dataset.classes)
print(f'Number of classes: {num_classes}')

### Split Datasets

In [ ]:
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Apply Training Transform to Training Dataset
train_dataset.dataset.transform = train_transform

### Create Data loaders

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

### Data Exploration

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    plt.imshow(torch.transpose(img, 0, 2).numpy())
    plt.show()

# Get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))

## Neutral Net

### NN Definition

In [ ]:
class FoodNet(nn.Module):
    def __init__(self, num_classes):  # Change num_classes to your dataset's number of classes
        super(FoodNet, self).__init__()
        # Load pre-trained MobileNet model
        self.mobilenet = models.mobilenet_v2(pretrained=True)
        # Replace the classifier layer to match the number of classes
        self.mobilenet.classifier[1] = nn.Linear(self.mobilenet.last_channel, num_classes)

    def forward(self, x):
        return self.mobilenet(x)

model = FoodNet(num_classes=num_classes).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

### Loss Function and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
best_val_loss = float('inf')
best_model_state = None

num_epochs = 10  

for epoch in range(num_epochs):
    model.train()
    # Training loop
    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    # Validation Loop
    model.eval()
    total = 0
    correct = 0
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total

    if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()

    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}%')



### Load the best model

In [ ]:
model.load_state_dict(best_model_state)

### Model Evaluation

In [ ]:
## Test the Model
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}%')

### Save Model

In [ ]:
torch.save(model.state_dict(), 'food_model, 256x256.pth')

## Test with your own image

In [ ]:
# Path to the test folder
test_folder = '../data/test'

# Define the transform
transform = transforms.Compose([
    transforms.Resize(resolution),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Ensure the model is in evaluation mode
model.eval()

# Iterate through each file in the test folder
for filename in os.listdir(test_folder):
    file_path = os.path.join(test_folder, filename)
    
    # Check if the file is an image
    if os.path.isfile(file_path) and file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
        image = Image.open(file_path)

        # Apply the transformation
        image = transform(image).unsqueeze(0)  # Add batch dimension

        # Predict with the model
        with torch.no_grad():
            image = image.to('cuda') if torch.cuda.is_available() else image
            outputs = model(image)
            _, predicted = torch.max(outputs.data, 1)
            predicted_class = dataset.classes[predicted.item()]

        # Print the result
        print(f'Image: {filename}, Predicted Class: {predicted_class}')

## Convert Model for Mobile Deployment (Tensorflow Lite)

### Convert to ONNX

In [ ]:
model.eval()
model.to('cpu')
dummy_input = torch.randn(1, 3, 256, 256)  # Adjust the shape to match your model's input
torch.onnx.export(model, dummy_input, "model.onnx", export_params=True, input_names=['input'], output_names=['output'])
## Load and Rename ONNX Model Inputs if Necessary
onnx_model = onnx.load("model.onnx")
print("Model Inputs: ", [inp.name for inp in onnx_model.graph.input])

# Define a mapping from old names to new names if needed
name_map = {"input.1": "input_1"}

# Initialize a list to hold the new inputs
new_inputs = []

# Iterate over the inputs and change their names if needed
for inp in onnx_model.graph.input:
    if inp.name in name_map:
        new_inp = helper.make_tensor_value_info(name_map[inp.name],
                                                inp.type.tensor_type.elem_type,
                                                [dim.dim_value for dim in inp.type.tensor_type.shape.dim])
        new_inputs.append(new_inp)
    else:
        new_inputs.append(inp)

# Clear the old inputs and add the new ones
onnx_model.graph.ClearField("input")
onnx_model.graph.input.extend(new_inputs)

# Go through all nodes in the model and replace the old input name with the new one
for node in onnx_model.graph.node:
    for i, input_name in enumerate(node.input):
        if input_name in name_map:
            node.input[i] = name_map[input_name]

# Save the renamed ONNX model
onnx.save(onnx_model, 'model_updated_256x256.onnx')

### Convert ONNX to TensorFlow

In [ ]:
# ## Convert Updated ONNX Model to TensorFlow
# onnx_model = onnx.load("model_updated.onnx")
# tf_rep = prepare(onnx_model)
# tf_rep.export_graph("model_tf")


### Convert TensorFlow Model to TensorFlow Lite

In [ ]:
# ## Convert TensorFlow Model to TensorFlow Lite
# converter = tf.lite.TFLiteConverter.from_saved_model("model_tf")  # Path to the SavedModel directory
# tflite_model = converter.convert()

# import json

# # Assuming 'dataset.classes' contains your class names
# class_names = dataset.classes

# # Save to a JSON file
# with open('class_names.json', 'w') as f:
#     json.dump(class_names, f)


### Test TensorFlowLite

In [ ]:
# # Load TFLite model and allocate tensors
# interpreter = tf.lite.Interpreter(model_path="model.tflite")
# interpreter.allocate_tensors()

# # Get input and output tensors
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()


# # Define the transform for test data
# test_transform = transforms.Compose([
#     transforms.Resize(resolution),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# # Iterate through each file in the test folder and run inference
# for filename in os.listdir(test_folder):
#     file_path = os.path.join(test_folder, filename)
    
#     # Check if the file is an image
#     if os.path.isfile(file_path) and file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
#         image = Image.open(file_path)

#         # Apply the transformation
#         image = test_transform(image).unsqueeze(0)  # Add batch dimension
#         image = image.numpy()

#         # Set input tensor
#         interpreter.set_tensor(input_details[0]['index'], image)

#         # Run inference
#         interpreter.invoke()

#         # Get output tensor
#         output_data = interpreter.get_tensor(output_details[0]['index'])
#         predicted_label = np.argmax(output_data)

#         # Assuming `dataset.classes` is the list of class names in the same order as used for training
#         class_names = dataset.classes

#         # Then in your inference loop
#         predicted_class_name = class_names[predicted_label]
#         print(f'Image: {filename}, Predicted Class: {predicted_class_name}')

